In [1]:
import numpy as np
import matplotlib.pyplot as plt
from sklearn.preprocessing import StandardScaler
import pandas as pd
from sklearn.decomposition import KernelPCA


In [2]:
def split_scalar(X,y):
    from sklearn.model_selection import train_test_split
    X_train, X_test, y_train, y_test = train_test_split(X, y, test_size = 0.2, random_state = 0)
    sc = StandardScaler()
    X_train = sc.fit_transform(X_train)
    X_test = sc.transform(X_test)
    return X_train,X_test,y_train,y_test


In [3]:
def Kernel_pca(X_train,X_test,n):
    kpca = KernelPCA(n_components = n,kernel='rbf')
    X_train_kpca = kpca.fit_transform(X_train)
    X_test_kpca = kpca.transform(X_test)
    return kpca,X_train_kpca,X_test_kpca


In [4]:
def r2_prediction(regressor,X_test_kpca,y_test):
        y_pred = regressor.predict(X_test_kpca)
        from sklearn.metrics import r2_score
        r2=r2_score(y_test,y_pred)
        return r2
    
def Linear(X_train_kpca,y_train,X_test_kpca):       
        # Fitting K-NN to the Training set
        from sklearn.linear_model import LinearRegression
        regressor = LinearRegression()
        regressor.fit(X_train_kpca, y_train)
        r2=r2_prediction(regressor,X_test_kpca,y_test)
        return  r2   
    
def svm_linear(X_train_kpca,y_train,X_test_kpca):
        from sklearn.svm import SVR
        regressor = SVR(kernel = 'linear')
        regressor.fit(X_train_kpca, y_train)
        r2=r2_prediction(regressor,X_test_kpca,y_test)
        return  r2  
    
def svm_NL(X_train_kpca,y_train,X_test_kpca):
        from sklearn.svm import SVR
        regressor = SVR(kernel = 'rbf')
        regressor.fit(X_train_kpca, y_train)
        r2=r2_prediction(regressor,X_test_kpca,y_test)
        return  r2  
    
def Decision(X_train_kpca,y_train,X_test_kpca):
        # Fitting K-NN to the Training setC
        from sklearn.tree import DecisionTreeRegressor
        regressor = DecisionTreeRegressor(random_state = 0)
        regressor.fit(X_train_kpca, y_train)
        r2=r2_prediction(regressor,X_test_kpca,y_test)
        return  r2  
     
def random(X_train_kpca,y_train,X_test_kpca):       
        # Fitting K-NN to the Training set
        from sklearn.ensemble import RandomForestRegressor
        regressor = RandomForestRegressor(n_estimators = 10, random_state = 0)
        regressor.fit(X_train_kpca, y_train)
        r2=r2_prediction(regressor,X_test_kpca,y_test)
        return  r2 
    
def kpca_merge(acclin,accrf,accdes,accsvml,accsvmnl): 
        rfedataframe=pd.DataFrame(index=['KPCA'],columns=['Logistic','Random','Decision','SVMl','SVMnl'])
        for number,idex in enumerate(rfedataframe.index):
            rfedataframe['Logistic'][idex]=acclin[number] 
            rfedataframe['Random'][idex]=accrf[number]
            rfedataframe['Decision'][idex]=accdes[number]
            rfedataframe['SVMl'][idex]=accsvml[number]
            rfedataframe['SVMnl'][idex]=accsvmnl[number]
        return rfedataframe

In [5]:
# Importing the dataset
dataset = pd.read_csv('Wine.csv')
X = dataset.iloc[:, 0:13].values
y = dataset.iloc[:, 13].values


In [6]:
X_train, X_test, y_train, y_test=split_scalar(X,y) 

# created a empty list to append the final value
acclin=[]
accsvml=[]
accsvmnl=[]
accdes=[]
accrf=[]

   
kpca,X_train_kpca,X_test_kpca,= Kernel_pca(X_train,X_test,5)

r2_lin=Linear(X_train_kpca,y_train,X_test_kpca)
acclin.append(r2_lin)

r2_sl=svm_linear(X_train_kpca,y_train,X_test_kpca)   
accsvml.append(r2_sl)

r2_NL=svm_NL(X_train_kpca,y_train,X_test_kpca)
accsvmnl.append(r2_NL)

r2_d=Decision(X_train_kpca,y_train,X_test_kpca)
accdes.append(r2_d)

r2_r=random(X_train_kpca,y_train,X_test_kpca)
accrf.append(r2_r)
    
result = kpca_merge(acclin,accsvml,accsvmnl,accdes,accrf)

In [7]:
result

,Logistic,Random,Decision,SVMl,SVMnl
KPCA,0.907981,0.897217,0.962011,0.945122,0.975854
